## Initialisation

In [98]:
#importing modules
import pandas as pd
from pathlib import Path

In [99]:
#forming the pathways to obtain the csv files
school_path = Path("..","Resources","schools_complete.csv")
student_path = Path("..","Resources","students_complete.csv")

In [100]:
#turning the school csv into a python dataframe
school_df = pd.read_csv(school_path)
school_df.head()

School ID            school_name         type  size   budget
0          0      Huang High School   Government  2917  1910635
1          1   Figueroa High School   Government  2949  1884411
2          2    Shelton High School  Independent  1761  1056600
3          3  Hernandez High School   Government  4635  3022020
4          4    Griffin High School  Independent  1468   917500

In [101]:
#turning the student csv into a python dataframe
student_df = pd.read_csv(student_path)
student_df.head()

Student ID     student_name gender  year        school_name  reading_score  \
0           0     Paul Bradley      M     9  Huang High School             96   
1           1     Victor Smith      M    12  Huang High School             90   
2           2  Kevin Rodriguez      M    12  Huang High School             41   
3           3    Richard Scott      M    12  Huang High School             89   
4           4       Bonnie Ray      F     9  Huang High School             87   

   maths_score  
0           94  
1           43  
2           76  
3           86  
4           69

In [102]:
#merging the two tables together
complete_data_df = pd.merge(school_df,student_df,how="left",on=["school_name","school_name"])
complete_data_df.head()

School ID        school_name        type  size   budget  Student ID  \
0          0  Huang High School  Government  2917  1910635           0   
1          0  Huang High School  Government  2917  1910635           1   
2          0  Huang High School  Government  2917  1910635           2   
3          0  Huang High School  Government  2917  1910635           3   
4          0  Huang High School  Government  2917  1910635           4   

      student_name gender  year  reading_score  maths_score  
0     Paul Bradley      M     9             96           94  
1     Victor Smith      M    12             90           43  
2  Kevin Rodriguez      M    12             41           76  
3    Richard Scott      M    12             89           86  
4       Bonnie Ray      F     9             87           69

### Local Government Area Summary

In [204]:
#forming a dataframe that contains a local government area summary
#number of unique schools
unique_schools = complete_data_df["school_name"].nunique()
#total number of students
students = len(complete_data_df.index)
#total budget
total_budget = school_df["budget"].sum()
#average math score
math_mean = complete_data_df["maths_score"].mean()
#average reading score
reading_mean = complete_data_df["reading_score"].mean()
#the percentage of students who pass maths
pass_math = complete_data_df.loc[complete_data_df["maths_score"] >= 50,"maths_score"].count()/students
#the percentage of students who pass reading
pass_read = complete_data_df.loc[complete_data_df["reading_score"] >= 50,"reading_score"].count()/students
#the percentage of students who pass maths and reading
pass_math_read = complete_data_df.loc[(complete_data_df["reading_score"] >= 50) & (complete_data_df["maths_score"] >= 50),"maths_score"].count()/students
#making the local government area summary dataframe
area_summary = pd.DataFrame([{"Unique Schools":unique_schools,"Total Students":students,"Total Budget":total_budget,"Average Math Score":math_mean,"Average Reading Score":reading_mean,"Pass Math":pass_math,"Pass Reading":pass_read,"Pass Math and Reading":pass_math_read}])
area_summary["Total Students"] = area_summary["Total Students"].map("{:,}".format)
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,}".format)
area_summary["Average Math Score"] = area_summary["Average Math Score"].map("{:.4f}%".format)
area_summary["Average Reading Score"] = area_summary["Average Reading Score"].map("{:.4f}%".format)
area_summary["Pass Math"] = area_summary["Pass Math"].map("{:,.4%}".format)
area_summary["Pass Reading"] = area_summary["Pass Reading"].map("{:,.4%}".format)
area_summary["Pass Math and Reading"] = area_summary["Pass Math and Reading"].map("{:,.4%}".format)
area_summary

Unique Schools Total Students Total Budget Average Math Score  \
0              15         39,170  $24,649,428           70.3382%   

  Average Reading Score Pass Math Pass Reading Pass Math and Reading  
0              69.9801%  86.0786%     84.4269%              72.8083%

### School Summary

In [104]:
#forming the basic information that will fill the dataframe
#forming an array that contains all the unique schools
school_name = complete_data_df.groupby(["school_name"])
#obtaining the school type
school_type = school_name["type"].first()
#obtaining the total students for each school
school_students = school_name["size"].count()
#obtaining the total school budget
school_budget = school_name["budget"].first()
#obtaining the per student budget
budg_per_stu = school_budget / school_students
#average math score per school
avg_math_school = school_name["maths_score"].mean()
#average reading score per school
avg_read_school = school_name["reading_score"].mean()
#percentage of students who pass maths per school
mapass_df = complete_data_df[complete_data_df["maths_score"] >= 50]
mapass_gp = mapass_df.groupby(["school_name"])
mapass_pcnt_school = mapass_gp["maths_score"].count() / school_students
#percentage of students who pass reading per school
readpass_df = complete_data_df[complete_data_df["reading_score"] >= 50]
readpass_gp = readpass_df.groupby(["school_name"])
readpass_pcnt_school = readpass_gp["reading_score"].count() / school_students
#percentage of students who pass both per school
totalpass_df = complete_data_df[(complete_data_df["maths_score"] >= 50) & (complete_data_df["reading_score"] >= 50)]
totalpass_gp = totalpass_df.groupby(["school_name"])
both_pass_pcnt_school = totalpass_gp["student_name"].count() / school_students

In [105]:
#forming a dictionary to make dataframe formation easier
sch_sum = {"School Type":school_type,"Total Students":school_students,"Total School Budget":school_budget,"Budget per Student":budg_per_stu,
           "Average Maths Score":avg_math_school,"Average Reading Score":avg_read_school,"Pass Math":mapass_pcnt_school,"Pass Reading":readpass_pcnt_school,
           "Overall Pass":both_pass_pcnt_school}
#making the actual dataframe based on the dictionary above. This dataframe will contain the raw numerical values for use later
per_school_summary_raw = pd.DataFrame(sch_sum)
#making the dataframe that will be formatted for the current situation
per_school_summary = per_school_summary_raw.copy()
#formatting the dataframe for clarity
per_school_summary.index.name = "School Name"
per_school_summary["Total Students"] = per_school_summary["Total Students"].map("{:,}".format)
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,}".format)
per_school_summary["Budget per Student"] = per_school_summary["Budget per Student"].map("${:,}".format)
per_school_summary["Average Maths Score"] = per_school_summary["Average Maths Score"].map("{:.4f}%".format)
per_school_summary["Average Reading Score"] = per_school_summary["Average Reading Score"].map("{:.4f}%".format)
per_school_summary["Pass Math"] = per_school_summary["Pass Math"].map("{:,.4%}".format)
per_school_summary["Pass Reading"] = per_school_summary["Pass Reading"].map("{:,.4%}".format)
per_school_summary["Overall Pass"] = per_school_summary["Overall Pass"].map("{:,.4%}".format)
per_school_summary

School Type Total Students Total School Budget  \
School Name                                                             
Bailey High School      Government          4,976          $3,124,928   
Cabrera High School    Independent          1,858          $1,081,356   
Figueroa High School    Government          2,949          $1,884,411   
Ford High School        Government          2,739          $1,763,916   
Griffin High School    Independent          1,468            $917,500   
Hernandez High School   Government          4,635          $3,022,020   
Holden High School     Independent            427            $248,087   
Huang High School       Government          2,917          $1,910,635   
Johnson High School     Government          4,761          $3,094,650   
Pena High School       Independent            962            $585,858   
Rodriguez High School   Government          3,999          $2,547,363   
Shelton High School    Independent          1,761          $1,056,600   
Thomas High School     Independent          1,635          $1,043,130   
Wilson High School     Independent          2,283          $1,319,574   
Wright High School     Independent          1,800          $1,049,400   

                      Budget per Student Average Maths Score  \
School Name                                                    
Bailey High School                $628.0            72.3529%   
Cabrera High School               $582.0            71.6572%   
Figueroa High School              $639.0            68.6985%   
Ford High School                  $644.0            69.0913%   
Griffin High School               $625.0            71.7881%   
Hernandez High School             $652.0            68.8749%   
Holden High School                $581.0            72.5831%   
Huang High School                 $655.0            68.9352%   
Johnson High School               $650.0            68.8431%   
Pena High School                  $609.0            72.0884%   
Rodriguez High School             $637.0            72.0478%   
Shelton High School               $600.0            72.0341%   
Thomas High School                $638.0            69.5817%   
Wilson High School                $578.0            69.1708%   
Wright High School                $583.0            72.0472%   

                      Average Reading Score Pass Math Pass Reading  \
School Name                                                          
Bailey High School                 71.0088%  91.6399%     87.3794%   
Cabrera High School                71.3595%  90.8504%     89.0743%   
Figueroa High School               69.0780%  81.6548%     82.8077%   
Ford High School                   69.5725%  82.4388%     82.2198%   
Griffin High School                71.2452%  91.2125%     88.4877%   
Hernandez High School              69.1864%  80.9493%     81.8770%   
Holden High School                 71.6604%  89.9297%     88.5246%   
Huang High School                  68.9105%  81.6935%     81.4535%   
Johnson High School                69.0393%  82.0626%     81.9786%   
Pena High School                   71.6133%  91.6840%     86.5904%   
Rodriguez High School              70.9360%  90.7977%     87.3968%   
Shelton High School                70.2578%  91.5389%     86.7121%   
Thomas High School                 69.7688%  83.8532%     82.6300%   
Wilson High School                 68.8769%  82.7858%     81.2965%   
Wright High School                 70.9694%  91.7778%     86.6667%   

                      Overall Pass  
School Name                         
Bailey High School        80.0844%  
Cabrera High School       80.7858%  
Figueroa High School      67.6501%  
Ford High School          67.4699%  
Griffin High School       81.3351%  
Hernandez High School     66.3646%  
Holden High School        78.9227%  
Huang High School         66.7124%  
Johnson High School       67.1918%  
Pena High School          79.2100%  
Rodriguez High School     79.4199%  
Shelton High School 

### Highest Performing Schools (By % Overall Passing)

In [106]:
#making a dataframe that sorts per_school_summary by the highest performing schools
top_schools_df = per_school_summary.sort_values("Overall Pass",ascending=False)
top_schools_df.head()                                                

School Type Total Students Total School Budget  \
School Name                                                             
Griffin High School    Independent          1,468            $917,500   
Cabrera High School    Independent          1,858          $1,081,356   
Bailey High School      Government          4,976          $3,124,928   
Wright High School     Independent          1,800          $1,049,400   
Rodriguez High School   Government          3,999          $2,547,363   

                      Budget per Student Average Maths Score  \
School Name                                                    
Griffin High School               $625.0            71.7881%   
Cabrera High School               $582.0            71.6572%   
Bailey High School                $628.0            72.3529%   
Wright High School                $583.0            72.0472%   
Rodriguez High School             $637.0            72.0478%   

                      Average Reading Score Pass Math Pass Reading  \
School Name                                                          
Griffin High School                71.2452%  91.2125%     88.4877%   
Cabrera High School                71.3595%  90.8504%     89.0743%   
Bailey High School                 71.0088%  91.6399%     87.3794%   
Wright High School                 70.9694%  91.7778%     86.6667%   
Rodriguez High School              70.9360%  90.7977%     87.3968%   

                      Overall Pass  
School Name                         
Griffin High School       81.3351%  
Cabrera High School       80.7858%  
Bailey High School        80.0844%  
Wright High School        79.7222%  
Rodriguez High School     79.4199%

### Lowest Performing Schools (By % Overall Passing)

In [107]:
#making a dataframe that sorts per_school_summary by the lowest performing schools
bottom_schools_df = per_school_summary.sort_values("Overall Pass")
bottom_schools_df.head()

School Type Total Students Total School Budget  \
School Name                                                             
Hernandez High School   Government          4,635          $3,022,020   
Huang High School       Government          2,917          $1,910,635   
Johnson High School     Government          4,761          $3,094,650   
Wilson High School     Independent          2,283          $1,319,574   
Ford High School        Government          2,739          $1,763,916   

                      Budget per Student Average Maths Score  \
School Name                                                    
Hernandez High School             $652.0            68.8749%   
Huang High School                 $655.0            68.9352%   
Johnson High School               $650.0            68.8431%   
Wilson High School                $578.0            69.1708%   
Ford High School                  $644.0            69.0913%   

                      Average Reading Score Pass Math Pass Reading  \
School Name                                                          
Hernandez High School              69.1864%  80.9493%     81.8770%   
Huang High School                  68.9105%  81.6935%     81.4535%   
Johnson High School                69.0393%  82.0626%     81.9786%   
Wilson High School                 68.8769%  82.7858%     81.2965%   
Ford High School                   69.5725%  82.4388%     82.2198%   

                      Overall Pass  
School Name                         
Hernandez High School     66.3646%  
Huang High School         66.7124%  
Johnson High School       67.1918%  
Wilson High School        67.4551%  
Ford High School          67.4699%

### Math Scores (By Year)

In [108]:
#organising a pandas series for each year level
ninth = complete_data_df[complete_data_df["year"] == 9]
tenth = complete_data_df[complete_data_df["year"] == 10]
eleventh = complete_data_df[complete_data_df["year"] == 11]
twelveth = complete_data_df[complete_data_df["year"] == 12]
#segregating each of the years by the school that the students are in
ninth_by_school = ninth.groupby(["school_name"])
tenth_by_school = tenth.groupby(["school_name"])
eleventh_by_school = eleventh.groupby(["school_name"])
twelveth_by_school = twelveth.groupby(["school_name"])

In [109]:
#collecting the average math scores for each year group
ninth_math_mean = ninth_by_school["maths_score"].mean()
tenth_math_mean = tenth_by_school["maths_score"].mean()
eleventh_math_mean = eleventh_by_school["maths_score"].mean()
twelveth_math_mean = twelveth_by_school["maths_score"].mean()
#forming a dictionary to contain all of these results
yearly_math = {"Year 9" : ninth_math_mean,"Year 10" : tenth_math_mean,"Year 11" : eleventh_math_mean,"Year 12" : twelveth_math_mean}
#forming the dataframe
yearly_math_df = pd.DataFrame(yearly_math)
#formatting the dataframe
yearly_math_df.index.name = "School Name"
yearly_math_df

Year 9    Year 10    Year 11    Year 12
School Name                                                      
Bailey High School     72.493827  71.897498  72.374900  72.675097
Cabrera High School    72.321970  72.437768  71.008299  70.604712
Figueroa High School   68.477804  68.331586  68.811001  69.325282
Ford High School       69.021609  69.387006  69.248862  68.617811
Griffin High School    72.789731  71.093596  71.692521  71.469178
Hernandez High School  68.586831  68.867156  69.154412  68.985075
Holden High School     70.543307  75.105263  71.640777  73.409639
Huang High School      69.081754  68.533246  69.431345  68.639316
Johnson High School    69.469286  67.990220  68.637730  69.287393
Pena High School       71.996364  72.396000  72.523438  71.187845
Rodriguez High School  71.940722  71.779808  72.364811  72.154626
Shelton High School    72.932075  72.506696  70.097087  72.331536
Thomas High School     69.234273  70.057007  69.657831  69.369822
Wilson High School     69.212361  69.455446  68.378965  69.787472
Wright High School     71.741176  72.179012  73.275862  70.848238

### Reading Scores (By Year)

In [110]:
#collecting the average reading scores for each year level
ninth_read_mean = ninth_by_school["reading_score"].mean()
tenth_read_mean = tenth_by_school["reading_score"].mean()
eleventh_read_mean = eleventh_by_school["reading_score"].mean()
twelveth_read_mean = twelveth_by_school["reading_score"].mean()
#forming a dictionary to contain all of these results
yearly_read = {"Year 9" : ninth_read_mean,"Year 10" : tenth_read_mean,"Year 11" : eleventh_read_mean,"Year 12" : twelveth_read_mean}
#forming the dataframe
yearly_read_df = pd.DataFrame(yearly_read)
#formatting the dataframe
yearly_read_df.index.name = "School Name"
yearly_read_df

Year 9    Year 10    Year 11    Year 12
School Name                                                      
Bailey High School     70.901920  70.848265  70.317346  72.195525
Cabrera High School    71.172348  71.328326  71.201245  71.856021
Figueroa High School   70.261682  67.677588  69.152327  69.082126
Ford High School       69.615846  68.988701  70.735964  68.849722
Griffin High School    72.026895  70.746305  72.385042  69.434932
Hernandez High School  68.477569  70.621842  68.418199  69.244136
Holden High School     71.598425  71.096491  73.310680  70.481928
Huang High School      68.670616  69.516297  68.740638  68.671795
Johnson High School    68.719286  69.295029  69.969115  67.992521
Pena High School       70.949091  72.324000  71.703125  71.513812
Rodriguez High School  70.902921  70.137500  71.424453  71.414449
Shelton High School    70.715094  69.879464  71.150485  69.070081
Thomas High School     69.672451  69.741093  70.749398  68.730769
Wilson High School     68.683043  68.412541  68.796327  69.888143
Wright High School     71.823529  71.386831  70.475862  69.821138

### Scores By School Spending

In [201]:
#forming the bins and labels to organise the dataset according to school spending
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
#creating a separate dataframe to conduct this business
school_spending_df = per_school_summary_raw.copy()
#binning the data
school_spending_df["spending_bins"] = pd.cut(school_spending_df["Budget per Student"], spending_bins, labels = labels)
#calculating the mean scores per spending range
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Pass Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Pass Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Overall Pass"].mean()
#forming the dataframe
spending_summary = pd.DataFrame({"Average Maths Score" : spending_math_scores,"Average Reading Score" : spending_reading_scores,
                                 "% Passing Math" : spending_passing_math,"% Passing Reading" : spending_passing_reading,
                                 "% Overall Passing": overall_passing_spending})
#formatting the dataframe then displaying it
spending_summary.index = ["<$585", "$585-630", "$630-645", "$645-680"]
spending_summary.index.name = "Spending Ranges (Per Student)"
spending_summary["% Passing Math"] = spending_summary["% Passing Math"] * 100
spending_summary["% Passing Reading"] = spending_summary["% Passing Reading"] * 100
spending_summary["% Overall Passing"] = spending_summary["% Overall Passing"] * 100
spending_summary["Average Maths Score"] = spending_summary["Average Maths Score"].map("{:.4f}%".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:.4f}%".format)
spending_summary["% Passing Math"] = spending_summary["% Passing Math"].map("{:.4f}%".format)
spending_summary["% Passing Reading"] = spending_summary["% Passing Reading"].map("{:.4f}%".format)
spending_summary["% Overall Passing"] = spending_summary["% Overall Passing"].map("{:.4f}%".format)
spending_summary

C:\Users\rashe\AppData\Local\Temp\ipykernel_18464\1286705241.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
C:\Users\rashe\AppData\Local\Temp\ipykernel_18464\1286705241.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
C:\Users\rashe\AppData\Local\Temp\ipykernel_18464\1286705241.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True

Average Maths Score Average Reading Score  \
Spending Ranges (Per Student)                                             
<$585                                    71.3646%              70.7166%   
$585-630                                 72.0659%              71.0313%   
$630-645                                 69.8548%              69.8388%   
$645-680                                 68.8844%              69.0454%   

                              % Passing Math % Passing Reading  \
Spending Ranges (Per Student)                                    
<$585                               88.8359%          86.3905%   
$585-630                            91.5188%          87.2924%   
$630-645                            84.6861%          83.7636%   
$645-680                            81.5685%          81.7697%   

                              % Overall Passing  
Spending Ranges (Per Student)                    
<$585                                  76.7215%  
$585-630                               79.8763%  
$630-645                               71.0050%  
$645-680                               66.7563%

### Scores By School Size

In [202]:
#forming a similar dataframe but rather than basing it on the budget per student, we will base it on school size
#forming the bins and labels
size_bins = [0, 1000, 2000, 5000]
size_labels = ["<1000", "1000-2000", "2000-5000"]
##creating a separate dataframe to conduct this business
school_sizescan_df = per_school_summary_raw.copy()
#binning the data
school_sizescan_df["size_bins"] = pd.cut(school_sizescan_df["Total Students"], size_bins, labels = size_labels)
#calculating the mean scores per school size
size_math_scores = school_sizescan_df.groupby(["size_bins"])["Average Maths Score"].mean()
size_reading_scores = school_sizescan_df.groupby(["size_bins"])["Average Reading Score"].mean()
size_passing_math = school_sizescan_df.groupby(["size_bins"])["Pass Math"].mean()
size_passing_reading = school_sizescan_df.groupby(["size_bins"])["Pass Reading"].mean()
overall_passing_size = school_sizescan_df.groupby(["size_bins"])["Overall Pass"].mean()
#forming the dataframe
size_summary = pd.DataFrame({"Average Maths Score" : size_math_scores,"Average Reading Score" : size_reading_scores,
                                 "% Passing Math" : size_passing_math,"% Passing Reading" : size_passing_reading,
                                 "% Overall Passing": overall_passing_size})
#formatting the dataframe then displaying it
size_summary.index = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
size_summary.index.name = "School Size"
size_summary["% Passing Math"] = size_summary["% Passing Math"] * 100
size_summary["% Passing Reading"] = size_summary["% Passing Reading"] * 100
size_summary["% Overall Passing"] = size_summary["% Overall Passing"] * 100
size_summary["Average Maths Score"] = size_summary["Average Maths Score"].map("{:.4f}%".format)
size_summary["Average Reading Score"] = size_summary["Average Reading Score"].map("{:.4f}%".format)
size_summary["% Passing Math"] = size_summary["% Passing Math"].map("{:.4f}%".format)
size_summary["% Passing Reading"] = size_summary["% Passing Reading"].map("{:.4f}%".format)
size_summary["% Overall Passing"] = size_summary["% Overall Passing"].map("{:.4f}%".format)
size_summary

C:\Users\rashe\AppData\Local\Temp\ipykernel_18464\1073821469.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = school_sizescan_df.groupby(["size_bins"])["Average Maths Score"].mean()
C:\Users\rashe\AppData\Local\Temp\ipykernel_18464\1073821469.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = school_sizescan_df.groupby(["size_bins"])["Average Reading Score"].mean()
C:\Users\rashe\AppData\Local\Temp\ipykernel_18464\1073821469.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=F

Average Maths Score Average Reading Score % Passing Math  \
School Size                                                                   
Small (<1000)                 72.3357%              71.6369%       90.8069%   
Medium (1000-2000)            71.4217%              70.7202%       89.8466%   
Large (2000-5000)             69.7518%              69.5761%       84.2528%   

                   % Passing Reading % Overall Passing  
School Size                                             
Small (<1000)               87.5575%          79.0663%  
Medium (1000-2000)          86.7141%          78.0398%  
Large (2000-5000)           83.3012%          70.2935%

### Scores By School Type

In [203]:
#forming a new dataframe analysing the same details as the above 2 dataframes but based on school type
#grouping according to the school type
stype = per_school_summary_raw.groupby("School Type")
#conducting the calculations
avg_math_score_type = stype["Average Maths Score"].mean()
avg_read_score_type = stype["Average Reading Score"].mean()
pass_math_type = stype["Pass Math"].mean()
pass_read_type = stype["Pass Reading"].mean()
overall_pass_type = stype["Overall Pass"].mean()

#forming the dataframe
type_summary = pd.DataFrame ({"Average Maths Score" : avg_math_score_type,"Average Reading Score" : avg_read_score_type,"% Passing Math" : pass_math_type,
                          "% Passing Reading" : pass_read_type,"% Overall Passing" : overall_pass_type})
#formatting the dataframe then displaying it
type_summary.index.name = "School Type"
type_summary["% Passing Math"] = type_summary["% Passing Math"] * 100
type_summary["% Passing Reading"] = type_summary["% Passing Reading"] * 100
type_summary["% Overall Passing"] = type_summary["% Overall Passing"] * 100
type_summary["Average Maths Score"] = type_summary["Average Maths Score"].map("{:.4f}%".format)
type_summary["Average Reading Score"] = type_summary["Average Reading Score"].map("{:.4f}%".format)
type_summary["% Passing Math"] = type_summary["% Passing Math"].map("{:.4f}%".format)
type_summary["% Passing Reading"] = type_summary["% Passing Reading"].map("{:.4f}%".format)
type_summary["% Overall Passing"] = type_summary["% Overall Passing"].map("{:.4f}%".format)
type_summary

Average Maths Score Average Reading Score % Passing Math  \
School Type                                                            
Government             69.8348%              69.6759%       84.4624%   
Independent            71.3688%              70.7189%       89.2040%   

            % Passing Reading % Overall Passing  
School Type                                      
Government           83.5876%          70.6990%  
Independent          86.2478%          76.9733%